In [81]:
import requests
import pandas as pd
from tqdm import tqdm
from utility import *


In [99]:
DISEASE_GENE_URI = "https://api.monarchinitiative.org/api/bioentity/disease/{}/genes"
DISEASE_VARIANT_URI = "https://api.monarchinitiative.org/api/bioentity/disease/{}/variants"

disease_path = "../../../data/benchmark_datasets/monarch/gwas_diseases.csv"

params = {}
params["rows"] = 2
params["direct"] = "true"
params["direct_taxon"] = "true"

In [74]:
disease_df = pd.read_csv(disease_path)
disease_df.columns = ["disease_id", "disease_name"]
disease_df.disease_id = disease_df.disease_id.apply(lambda x:x.split('"')[1])
disease_df

,disease_id,disease_name
0,DOID:0080630,"""B-lymphoblastic leukemia/lymphoma"""
1,DOID:8893,"""psoriasis"""
2,DOID:1712,"""aortic valve stenosis"""
3,DOID:13189,"""gout"""
4,DOID:5418,"""schizoaffective disorder"""
...,...,...
249,DOID:1380,"""endometrial cancer"""
250,DOID:6039,"""uveal melanoma"""
251,DOID:10538,"""gastric fundus cancer"""
252,DOID:0111910,"""spermatogenic failure"""


In [107]:
def get_api_resp(URI, params=None):
    if params:
        return requests.get(URI, params=params)
    else:
        return requests.get(URI)
    
def get_association(URI, disease_id, params, object_attribute="label"):
    URI_ = URI.format(disease_id)
    resp = get_api_resp(URI_, params=params)
    if resp.status_code == 200:
        response = resp.json()
        associations = response["associations"]
        object_list = []
        for item in associations:
            object_list.append(item["object"][object_attribute])
        df = pd.DataFrame(object_list, columns=["object"])
        df["subject"] = disease_id
        return df
    else:
        return None
    

In [71]:
edge_df_list = []

for index, row in tqdm(disease_df.iterrows()):
    edge_df_list.append(get_association(DISEASE_GENE_URI, row["disease_id"], params))

edge_df = pd.concat(edge_df_list, ignore_index=True)
edge_df = pd.merge(edge_df, disease_df, left_on="subject", right_on="disease_id").drop(["subject", "disease_id"], axis=1)
edge_df.disease_name = edge_df.disease_name.apply(lambda x:x.split('"')[1])

data_mcq_disease_gene = create_mcq(edge_df, "disease_name", "object", "Gene", "is associated with")

254it [01:46,  2.38it/s]


In [108]:
edge_df_list_2 = []

for index, row in tqdm(disease_df.iterrows()):
    edge_df_list_2.append(get_association(DISEASE_VARIANT_URI, row["disease_id"], params, object_attribute="id"))

edge_df_2 = pd.concat(edge_df_list_2, ignore_index=True)
edge_df_2 = pd.merge(edge_df_2, disease_df, left_on="subject", right_on="disease_id").drop(["subject", "disease_id"], axis=1)
edge_df_2.disease_name = edge_df_2.disease_name.apply(lambda x:x.split('"')[1])
edge_df_2.object = edge_df_2.object.apply(lambda x:x.split("dbSNP:")[-1])

254it [01:42,  2.47it/s]


In [114]:
data_mcq_disease_variant = create_mcq(edge_df_2, "disease_name", "object", "Variant", "is associated with")


In [121]:
data_mcq_monarch = pd.concat([data_mcq_disease_gene, data_mcq_disease_variant], ignore_index=True)


In [122]:
data_mcq_monarch.to_csv("../../../data/benchmark_datasets/test_questions_two_hop_mcq_from_monarch.csv", index=False, header=True)


In [124]:
data_mcq_disease_variant

,disease_pair,correct_node,negative_samples,disease_1,disease_2,text
1705,"(breast carcinoma, male breast cancer)",rs3803662,"rs17045031, rs170020, rs2115691, rs4728142",breast carcinoma,male breast cancer,"Out of the given list, which Variant is associ..."
2674,"(abdominal aortic aneurysm, thoracic aortic an...",rs595244,"rs170884, rs59325236, rs219780, rs4459895",abdominal aortic aneurysm,thoracic aortic aneurysm,"Out of the given list, which Variant is associ..."
2994,"(colon carcinoma, melanoma)",ClinVarVariant:13961,"rs2391769, rs11209026, rs452932, rs2857595",colon carcinoma,melanoma,"Out of the given list, which Variant is associ..."
3092,"(colon carcinoma, hepatocellular carcinoma)",ClinVarVariant:13652,"rs2129979, rs10490924, rs167769, rs2143571",colon carcinoma,hepatocellular carcinoma,"Out of the given list, which Variant is associ..."
3106,"(colon carcinoma, lung carcinoma)",ClinVarVariant:13961,"MGI:3817496, MGI:2450162, ClinVarVariant:14163...",colon carcinoma,lung carcinoma,"Out of the given list, which Variant is associ..."
3147,"(Kawasaki disease, inflammatory bowel disease)",rs1801274,"MGI:2655190, rs670142, rs2736100, rs170020",Kawasaki disease,inflammatory bowel disease,"Out of the given list, which Variant is associ..."
5839,"(inflammatory bowel disease, Kawasaki disease)",rs1801274,"ClinVarVariant:376659, rs78378222, rs16948876,...",inflammatory bowel disease,Kawasaki disease,"Out of the given list, which Variant is associ..."
6968,"(triple-receptor negative breast cancer, uteri...",rs78378222,"rs4543123, ClinVarVariant:18061, rs11015915, r...",triple-receptor negative breast cancer,uterine fibroid,"Out of the given list, which Variant is associ..."
7774,"(vitiligo, rheumatoid arthritis)",rs2476601,"rs3784262, rs9427573, rs72755233, FlyBase:FBal...",vitiligo,rheumatoid arthritis,"Out of the given list, which Variant is associ..."
8744,"(male breast cancer, breast carcinoma)",rs3803662,"ClinVarVariant:18061, rs10124723, rs10127000, ...",male breast cancer,breast carcinoma,"Out of the given list, which Variant is associ..."
